# OrientDB tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [3]:
import pyorient

In [4]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("localhost", 2424)
session_id = client.connect("root", ROOT_PASSWORD)

In [5]:
print(client.db_list())

{{'databases': {}}}


## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [6]:
# Create the database 'gods' with GRAPH_DATABASE type and MEMORY_STORAGE_TYPE
if not client.db_exists("gods", pyorient.STORAGE_TYPE_MEMORY):
    client.db_create("gods", "graph", "memory")

In [7]:
print(client.db_list())

{{'databases': {'gods': 'memory:gods'}}}


**Q:** Connect your pyorient client to the `gods` database.

In [8]:
# Connect to the 'gods' database
client.db_open("gods", "root", ROOT_PASSWORD)

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [9]:
# Create a new vertex with the content {name: 'Zeus', symbol: 'thunder'}
query = "CREATE VERTEX V SET name = 'Zeus', symbol = 'thunder'"

# Execute the query
result = client.command(query)

# Print the result
print("Vertex created:", result)

Vertex created: [<pyorient.otypes.OrientRecord object at 0x7b586589e350>]


You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [10]:
# Query to select all vertices from the 'V' table
query = "SELECT * FROM V"

# Execute the query and store the result
result = client.command(query)

# Print the result
print("All current vertices:")
for vertex in result:
    print(vertex.oRecordData)

All current vertices:
{'name': 'Zeus', 'symbol': 'thunder'}


**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [11]:
# Create new vertices with the specified content
queries = [
    "CREATE VERTEX V SET name = 'Héra', symbol = 'tiara'",
    "CREATE VERTEX V SET name = 'Poséidon', symbol = 'trident'",
    "CREATE VERTEX V SET name = 'Athena', symbol = 'helmet'",
    "CREATE VERTEX V SET name = 'Arès', symbol = 'weapons'"
]

# Execute each query to create the vertices
for query in queries:
    result = client.command(query)

In [12]:
query = "SELECT * FROM V"
result = client.command(query)
for vertex in result:
    print(vertex.oRecordData)

{'name': 'Zeus', 'symbol': 'thunder'}
{'name': 'Poséidon', 'symbol': 'trident'}
{'name': 'Arès', 'symbol': 'weapons'}
{'name': 'Héra', 'symbol': 'tiara'}
{'name': 'Athena', 'symbol': 'helmet'}


**Q:** Display all vertices with name = `Arès`

In [13]:
# Query to select all vertices with name = 'Arès'
query = "SELECT * FROM V WHERE name = 'Arès'"

# Execute the query and store the result
result = client.command(query)

# Print the result
for vertex in result:
    print(vertex.oRecordData)


{'name': 'Arès', 'symbol': 'weapons'}


**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [14]:
# Query to create an edge from Zeus to Poséidon with kind = 'sibling' using inline SELECT
create_edge_query = """
CREATE EDGE 
FROM (SELECT FROM V WHERE name = 'Zeus') 
TO (SELECT FROM V WHERE name = 'Poséidon') 
SET kind = 'sibling'
"""

# Execute the edge creation query
client.command(create_edge_query)

**Q:** Redisplay all vertices, discuss.

In [15]:
# Execute the query to display all vertices in the 'gods' database
vertices_query = "SELECT FROM V"
vertices = client.command(vertices_query)

# Print the results (displaying all vertices)
for vertex in vertices:
    print(vertex.oRecordData) 

{'name': 'Zeus', 'symbol': 'thunder', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdade50>}
{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdb1050>}
{'name': 'Arès', 'symbol': 'weapons'}
{'name': 'Héra', 'symbol': 'tiara'}
{'name': 'Athena', 'symbol': 'helmet'}


**Q:** Display all edges. They are contained in the class `E`

In [16]:
# Execute the query to display all edges in the 'gods' database (edges are contained in the class 'E')
edges_query = "SELECT FROM E"
edges = client.command(edges_query)

# Print the results (displaying all edges)
for edge in edges:
    print(edge.oRecordData)  

{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x7b583fdb1c90>, 'in': <pyorient.otypes.OrientRecordLink object at 0x7b583fdb1d50>}


Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [17]:
# Création de plusieurs arêtes (edges) entre les divinités grecques
edges_to_create = [
    ("Zeus", "Héra", "sibling"),
    ("Zeus", "Arès", "father"),
    ("Zeus", "Athena", "father"),
    ("Héra", "Arès", "mother"),
    ("Héra", "Zeus", "sibling"),
    ("Poséidon", "Zeus", "sibling"),
]

for from_name, to_name, kind in edges_to_create:
    client.command(f"""
        CREATE EDGE 
        FROM (SELECT FROM V WHERE name = '{from_name}') 
        TO (SELECT FROM V WHERE name = '{to_name}') 
        SET kind = '{kind}'
    """)

### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [18]:
# Define the query to fetch all outgoing vertices from Zeus
outgoing_query = "SELECT expand(out()) FROM V WHERE name = 'Zeus'"

# Execute the query
outgoing_vertices = client.command(outgoing_query)

# Display the results (expanded vertices connected and outgoing from Zeus)
for v in outgoing_vertices:
    print(v.oRecordData)  


{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdb71d0>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdb7ad0>}
{'out_': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdb7bd0>, 'name': 'Héra', 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdb7d10>}
{'in_': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdb7e10>, 'name': 'Arès', 'symbol': 'weapons'}
{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdbb110>}


**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [19]:
# Define the query to fetch all vertices that have a father
father_vertices_query = "SELECT expand(in) from E where kind='father'"
# Execute the query
father_vertices = client.command(father_vertices_query)

# Display the results (vertices with a father)
for vertex in father_vertices:
    print(vertex.oRecordData) 


{'in_': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdb7610>, 'name': 'Arès', 'symbol': 'weapons'}
{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdbb9d0>}


**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [20]:
# Define the query to fetch all vertices that are mothers
mother_vertices_query = "SELECT FROM V WHERE @rid IN (SELECT out FROM E WHERE kind = 'mother')"

# Execute the query
mother_vertices = client.command(mother_vertices_query)

# Display the results (vertices that are mothers)
for vertex in mother_vertices:
    print(vertex.oRecordData) 


{'out_': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdbbf50>, 'name': 'Héra', 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdbc090>}


**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [21]:
# Define the query to fetch the brothers and sisters of Zeus
siblings_query = "SELECT expand(in) FROM E WHERE kind = 'sibling' and out.name = 'Zeus'"

# Execute the query
siblings = client.command(siblings_query)

# Display the results (brothers and sisters of Zeus)
for sibling in siblings:
    print(sibling.oRecordData) 


{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdbc710>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdbc790>}
{'name': 'Héra', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdbc8d0>, 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdbc9d0>}


## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

In [22]:
from datetime import datetime   
# Step 1: Create the eCommerce Database
if not client.db_exists("eCommerce", pyorient.STORAGE_TYPE_MEMORY):
    client.db_create("eCommerce", "graph", "memory")

# Step 2: Create the necessary classes
client.command("CREATE CLASS Product EXTENDS V")  # Product class
client.command("CREATE PROPERTY Product.name STRING")  # name field for Product

client.command("CREATE CLASS Person EXTENDS V")  # Person class
client.command("CREATE PROPERTY Person.nickname STRING")  # nickname field for Person

client.command("CREATE CLASS Purchase EXTENDS E")  # Purchase edge class
client.command("CREATE PROPERTY Purchase.purchase_date DATETIME")  # purchase_date field for Purchase

# Step 3: Verify the classes
# Fetch all classes from the database
classes = client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)")

# Display the classes
for class_name in classes:
    print(class_name.oRecordData)


{'name': 'OSchedule'}
{'name': 'Person'}
{'name': 'OFunction'}
{'name': 'ORestricted'}
{'name': 'Purchase'}
{'name': 'OIdentity'}
{'name': 'OUser'}
{'name': 'OTriggered'}
{'name': 'V'}
{'name': 'Product'}
{'name': 'OSequence'}
{'name': 'E'}
{'name': 'ORole'}


**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

In [23]:
# Create products with specified names
product_names = ['spaghetti', 'bolognese sauce', 'cheese', 'apple']

# Loop through the product names and create a product vertex for each
for name in product_names:
    query = f"CREATE VERTEX Product SET name = '{name}'"
    client.command(query)

# Verify by printing the created products
products = client.command("SELECT FROM Product")
for product in products:
    print(product.oRecordData)  


{'name': 'spaghetti'}
{'name': 'cheese'}
{'name': 'bolognese sauce'}
{'name': 'apple'}


**Q:** Create the following people: `peter`, `meredith`.

In [24]:
people = ['peter', 'meredith']
for nickname in people:
    client.command(f"CREATE VERTEX Person SET nickname = '{nickname}'")

**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016


In [25]:
purchases = [
    ("peter", ["spaghetti", "cheese"], "2016-01-20"),
    ("meredith", ["cheese", "apple", "bolognese sauce"], "2016-01-22"),
    ("peter", ["spaghetti", "bolognese sauce"], "2016-01-27"),
]
for person, products, date in purchases:
    for product in products:
        client.command(f"""
            CREATE EDGE Purchase 
            FROM (SELECT FROM Person WHERE nickname = '{person}') 
            TO (SELECT FROM Product WHERE name = '{product}') 
            SET date = '{date}'
        """)


In [26]:
query = """
SELECT 
    out.nickname as buyer, 
    in.name as product, 
    date 
FROM Purchase
"""

results = client.command(query)
for r in results:
    print(r.oRecordData)

{'buyer': 'peter', 'product': 'spaghetti', 'date': '2016-01-20'}
{'buyer': 'meredith', 'product': 'cheese', 'date': '2016-01-22'}
{'buyer': 'meredith', 'product': 'bolognese sauce', 'date': '2016-01-22'}
{'buyer': 'peter', 'product': 'bolognese sauce', 'date': '2016-01-27'}
{'buyer': 'peter', 'product': 'cheese', 'date': '2016-01-20'}
{'buyer': 'meredith', 'product': 'apple', 'date': '2016-01-22'}
{'buyer': 'peter', 'product': 'spaghetti', 'date': '2016-01-27'}


**Q:** Who bought Bolognese sauce?

In [27]:
buyers = client.command("""
    SELECT FROM Person 
    WHERE @rid IN (
        SELECT out FROM Purchase 
        WHERE in IN (
            SELECT FROM Product WHERE name = 'bolognese sauce'
        )
    )
""")

for buyer in buyers:
    print(buyer.oRecordData)

{'out_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdbccd0>, 'nickname': 'peter'}
{'out_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x7b583fdbcf90>, 'nickname': 'meredith'}


**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

In [28]:
results = client.command("""
    SELECT expand(out('Purchase')) 
    FROM Person 
    WHERE @rid IN (
        SELECT out FROM Purchase 
        WHERE in IN (
            SELECT FROM Product WHERE name = 'bolognese sauce'
        )
    )
""")

# Utilise un set pour éviter les doublons
unique_products = set()

for product in results:
    name = product.oRecordData.get('name')
    if name and name != 'bolognese sauce':
        unique_products.add(name)

# Affiche les noms sans doublon
for name in unique_products:
    print(name)

spaghetti
apple
cheese


## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.